In [1]:
from theano.sandbox import cuda

%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using TensorFlow backend.


# Import corpus

In [2]:
notre_dame = "http://www.gutenberg.org/cache/epub/19657/pg19657.txt"
lesmiserable1 = "http://www.gutenberg.org/cache/epub/17489/pg17489.txt"

notredame_path = get_file('nd.txt', origin=str(notre_dame))
import codecs

text = codecs.open(notredame_path, "r", "utf-8").read()
print('corpus length:', len(text))
print(text[1:100])

corpus length: 1087507
The Project Gutenberg EBook of Notre-Dame de Paris, by Victor Hugo

This eBook is for the use of 


In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)
for i in chars[-6:]: print(i)

total chars: 115
ï
ñ
ô
ù
û
﻿


In [4]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [5]:
idx = [char_indices[c] for i,c in enumerate(text)]
idx[1:10]

[49, 66, 63, 2, 45, 76, 73, 68, 63]

In [6]:
"".join(indices_char[i] for i in idx[1:50])

u'The Project Gutenberg EBook of Notre-Dame de Pari'

# Simple RNN 

In [7]:
embedding_size = 42
hidden_layers = 256
character_set = 8
vocab_size = len(chars)+1

In [8]:
model=Sequential([
        Embedding(vocab_size, embedding_size, input_length=character_set),
        SimpleRNN(hidden_layers, activation='relu', inner_init='identity'),
        Dense(vocab_size, activation='softmax')
    ])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 8, 42)         4830        embedding_input_1[0][0]          
____________________________________________________________________________________________________
simplernn_1 (SimpleRNN)          (None, 256)           76544       embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 115)           29555       simplernn_1[0][0]                
Total params: 110,929
Trainable params: 110,929
Non-trainable params: 0
____________________________________________________________________________________________________


In [9]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())


## Preprocess input and output

In [10]:
type(idx[0])

int

In [11]:
#  from https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py
char_in = []
char_out = []
for i in range(0, len(text) - 1 - character_set, character_set):
    char_in.append([char_indices[c] for c in text[i: i + character_set]])
    char_out.append(char_indices[text[i + character_set]])
print('nb sequences:', len(char_in))

sequences[1]
# x= np.zeros(character_set,)
# y =[]
# for i, seq in enumerate(sequences):
#     for j, char in enumerate(seq):
#         x[]


NameError: name 'sequences' is not defined

## Fit

In [17]:
model.optimizer.lr=0.0001
model.fit(char_in, char_out, batch_size=64, nb_epoch=3)

Epoch 1/3
135938/135938 [==============================] - 24s - loss: 1.3895    
Epoch 2/3
135938/135938 [==============================] - 24s - loss: 1.3795    
Epoch 3/3
135938/135938 [==============================] - 25s - loss: 1.3686    


In [58]:
def get_nexts_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arr = np.array(idxs)[np.newaxis,:]
    p = model.predict(arr)[0]
    return indices_char[np.argmax(p)]
#     print(p)b
#     return [chars[np.argmax(o)] for o in p]

In [20]:
import random
i_space = [i for i, x in enumerate(text) if x == " "] 
for i in range(3):
    len_test = 40
    start_index = random.randint(0, len(i_space))
    sentence = text[i_space[start_index]:i_space[start_index]+len_test]
    constructed_sentence = sentence[0:character_set]
    next_seed = constructed_sentence
    print("seed: %s, original: %s" % (constructed_sentence, sentence))
    for i in range(len_test-character_set):
        next_letter = get_nexts_keras(next_seed)
        next_seed = next_seed[1:]+next_letter
        constructed_sentence = constructed_sentence + next_letter
    print("constructed: %s" % constructed_sentence)
    



seed:  de temp, original:  de temps en temps quand
Gringoire les 


KeyError: 730

### Sequential model

In [8]:
embedding_size = 42
hidden_layers = 256
character_set = 25
vocab_size = len(chars)+1

model=Sequential([
        Embedding(vocab_size, embedding_size, input_length=character_set),
        SimpleRNN(hidden_layers, return_sequences=True, activation='relu', inner_init='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax'))
    ])
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
model.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 25, 42)        4830        embedding_input_1[0][0]          
____________________________________________________________________________________________________
simplernn_1 (SimpleRNN)          (None, 25, 256)       76544       embedding_1[0][0]                
____________________________________________________________________________________________________
timedistributed_1 (TimeDistribut (None, 25, 115)       29555       simplernn_1[0][0]                
Total params: 110,929
Trainable params: 110,929
Non-trainable params: 0
____________________________________________________________________________________________________


In [9]:
#  from https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py
char_in = []
char_out = []
for i in range(0, len(text) - 1 - character_set, character_set):
    char_in.append([char_indices[c] for c in text[i: i + character_set]])
    char_out.append([char_indices[c] for c in text[i+1: i + 1 + character_set]])

print('nb sequences:', len(char_in))
print(char_in[1:3])
print(char_out[1:3])
char_out_3d = np.asarray(np.stack([char_out],axis=2))
print(char_out_3d.shape)





nb sequences: 43500
[[73, 73, 69, 2, 73, 64, 2, 43, 73, 78, 76, 63, 13, 33, 59, 71, 63, 2, 62, 63, 2, 45, 59, 76, 67], [77, 12, 2, 60, 83, 2, 51, 67, 61, 78, 73, 76, 2, 37, 79, 65, 73, 1, 0, 1, 0, 49, 66, 67, 77]]
[[73, 69, 2, 73, 64, 2, 43, 73, 78, 76, 63, 13, 33, 59, 71, 63, 2, 62, 63, 2, 45, 59, 76, 67, 77], [12, 2, 60, 83, 2, 51, 67, 61, 78, 73, 76, 2, 37, 79, 65, 73, 1, 0, 1, 0, 49, 66, 67, 77, 2]]
(43500, 25, 1)


In [16]:
model.lr=0.0001
model.fit(char_in, char_out_3d, batch_size=64, nb_epoch=3)

Epoch 1/3
43500/43500 [==============================] - 28s - loss: 1.4546    
Epoch 2/3
43500/43500 [==============================] - 28s - loss: 1.4500    
Epoch 3/3
43500/43500 [==============================] - 28s - loss: 1.4456    


In [24]:
def get_nexts_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arr = np.array(idxs)[np.newaxis,:]
    p = model.predict(arr)[0]

    return indices_char[np.argmax(p[7])]

In [28]:
def gen_sentence(num_sent):
    import random
    i_space = [i for i, x in enumerate(text) if x == " "] 
    for i in range(num_sent):
        len_test = 40
        seed_length = 8
        start_index = random.randint(0, len(i_space))
        sentence = text[i_space[start_index]:i_space[start_index]+len_test]
        constructed_sentence = sentence[0:character_set]
        next_seed = constructed_sentence
        print("seed: %s, original: %s" % (constructed_sentence, sentence))
        for i in range(len_test-character_set):
            next_letter = get_nexts_keras(next_seed)
            next_seed = next_seed[1:]+next_letter
            constructed_sentence = constructed_sentence + next_letter
        print("constructed: %s" % constructed_sentence)


In [ ]:
gen_sentence(3)

# Stateful RNN

In [7]:
embedding_size = 42
hidden_layers = 256
character_set = 25
vocab_size = len(chars)+1
batch_size = 64

model=Sequential([
        Embedding(vocab_size, embedding_size, input_length=character_set, batch_input_shape=(batch_size,character_set)),
        BatchNormalization(),
        LSTM(hidden_layers, return_sequences=True, stateful=True),
        TimeDistributed(Dense(vocab_size, activation='softmax'))
    ])

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
model.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (64, 25, 42)          4830        embedding_input_1[0][0]          
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (64, 25, 42)          168         embedding_1[0][0]                
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (64, 25, 256)         306176      batchnormalization_1[0][0]       
____________________________________________________________________________________________________
timedistributed_1 (TimeDistribut (64, 25, 115)         29555       lstm_1[0][0]                     
Total params: 340,729
Trainable params: 340,645
Non-trainable params: 84
__________________

In [8]:
#  from https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py
char_in = []
char_out = []
for i in range(0, len(text) - 1 - character_set, character_set):
    char_in.append([char_indices[c] for c in text[i: i + character_set]])
    char_out.append([char_indices[c] for c in text[i+1: i + 1 + character_set]])

print('nb sequences:', len(char_in))
## truncate to multiple batch size
mult_bs = (len(char_in)//batch_size)*batch_size
print(mult_bs)
print(len(char_in[:mult_bs]))
print(char_in[1:3])
print(char_out[1:3])
char_out_3d = np.asarray(np.stack([char_out[:mult_bs]],axis=2))
char_in = char_in[:mult_bs]
print(char_out_3d.shape)



nb sequences: 43500
43456
43456
[[73, 73, 69, 2, 73, 64, 2, 43, 73, 78, 76, 63, 13, 33, 59, 71, 63, 2, 62, 63, 2, 45, 59, 76, 67], [77, 12, 2, 60, 83, 2, 51, 67, 61, 78, 73, 76, 2, 37, 79, 65, 73, 1, 0, 1, 0, 49, 66, 67, 77]]
[[73, 69, 2, 73, 64, 2, 43, 73, 78, 76, 63, 13, 33, 59, 71, 63, 2, 62, 63, 2, 45, 59, 76, 67, 77], [12, 2, 60, 83, 2, 51, 67, 61, 78, 73, 76, 2, 37, 79, 65, 73, 1, 0, 1, 0, 49, 66, 67, 77, 2]]
(43456, 25, 1)


In [34]:
def gen_sentence(num_sent):
    import random
    i_space = [i for i, x in enumerate(text) if x == " "] 
    for i in range(num_sent):
        len_test = 40
        seed_length = 8
        start_index = random.randint(0, len(i_space))
        sentence = text[i_space[start_index]:i_space[start_index]+len_test]
        print("----> seed: %s, original: %s" % (sentence[0:character_set], sentence))

        for temp in [-1, 0.2, 0.5, 1, 1.2]:
            constructed_sentence = sentence[0:character_set]
            next_seed = constructed_sentence
            print(" __ temp %0.1f" % temp)
            for i in range(len_test-character_set):
                next_letter = get_nexts_keras(next_seed, temp)
                next_seed = next_seed[1:]+next_letter
                constructed_sentence = constructed_sentence + next_letter
            print("constructed: %s" % constructed_sentence)

In [26]:
# Selectiveness from https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)    
    probas = np.random.multinomial(1, preds, 1)
#     print("max prob %d vs max preds %d" % (np.argmax(probas),np.argmax(preds)))
    return np.argmax(probas)
 

In [32]:
def get_nexts_keras(inp, temperature= -1 ):
    idxs = [char_indices[c] for c in inp]
    arr = np.array(idxs)[np.newaxis,:]
    arr_b = np.repeat(arr, batch_size, axis=0)
    p = model.predict(arr_b, batch_size=batch_size)[0]
    if temperature < 0:
        return indices_char[np.argmax(p[0])]
    else:
        return indices_char[sample(p[0],temperature)]

In [ ]:
model.lr=0.0001
for i in range(20):
    print ("=== iteration %d ===" % i)
    model.fit(char_in, char_out_3d, batch_size=64, nb_epoch=5)
    gen_sentence(1)

=== iteration 0 ===
Epoch 1/5
43456/43456 [==============================] - 106s - loss: 1.5005   
Epoch 2/5
43456/43456 [==============================] - 117s - loss: 1.4929   
Epoch 3/5
 7232/43456 [===>..........................] - ETA: 83s - loss: 1.4874

In [ ]:
model.save_weights('./results/lstm_statefull_victorhugo.h5')